In [0]:
!uname -a

In [0]:
cd ~/ipynb/gypsy_moth/

In [0]:
import scandir
import os
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import dill
import random
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri as p2r
p2r.activate()
r = ro.r
from collections import defaultdict
from sklearn import preprocessing
import scipy as sp
import shutil
import pickle
from utils import read_df, save_df
import iterlib
from IPython.display import display
from collections import Counter
from ipyparallel import Client

In [0]:
analysis_dir_notimp = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni/"
analysis_dir_imp = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/"

In [0]:
analysis_dir = [analysis_dir_notimp, analysis_dir_imp]

In [0]:
%%R
library(sp)
library(raster)

In [0]:
raster = r("raster")
extract = r("extract")

gps = {'QC32':[47.2509807, -79.4060515],
      'QC93': [46.90826, -70.8061075],
      'NC': [36.449125, -76.024672],
      'NY': [42.897768, -74.094761],
      'VA1': [38.657615, -77.463603],
      'VA2': [38.857470, -77.695003]}
gps_df = pd.DataFrame(gps).T
gps_df.columns = ['lat','lon']

latlon = pandas2ri.py2ri(gps_df[['lon', 'lat']])

bioclim_dir = "/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/bioclim_13"
bioclim = !ls {bioclim_dir}/*.bil
bioclim = sorted(bioclim)
bioclim_df = pd.DataFrame(gps_df)
for b in bioclim:
    rast = raster(b)
    bio = os.path.basename(b).split("_")[0].upper()
    vals = pd.DataFrame(pandas2ri.ri2py(extract(rast, latlon)))
    vals.index = bioclim_df.index
    vals.columns = [bio]
    bioclim_df = bioclim_df.join(vals)
bioclim_df = bioclim_df.sort_index()

In [0]:
bioclim_df

In [0]:
for d in analysis_dir:
    save_df(d, "bioclim_df", bioclim_df)

In [0]:
z12_swapped = []
for d in analysis_dir:
    z12_swapped.append(read_df(d, "z12_swapped"))

In [0]:
z12_swapped[1].head()

In [0]:
pop_allele_freqs = []
for d in analysis_dir:
    infile = os.path.join(d, "pop_allele_data.pkl")
    paf = pickle.load(open(infile, "rb"))
    pop_allele_freqs.append(paf)

In [0]:
snpsfile_df = []
for i, paf in enumerate(pop_allele_freqs):
    pops = sorted(paf)
    paf_data = defaultdict(defaultdict)
    for popn in pops:
        af = pd.DataFrame(paf[popn])
        for snp in af:
            paf_data["%s_1" % snp][popn] = af.ix["P",snp]
            paf_data["%s_2" % snp][popn] = af.ix["Q",snp]  
    df = pd.DataFrame(paf_data).T
    df['blank'] = ""
    snpsfile_df.append(df)

In [0]:
environ_cols = snpsfile_df[0].columns.drop("blank")

In [0]:
environ_cols

In [0]:
bioclim = [read_df(x, "bioclim_df") for x in analysis_dir]

In [0]:
print(bioclim[1].to_csv())

In [0]:
bioclim = [bioclim[1]]
bioclim = [x[[y for y in x if 'BIO' in y]] for x in bioclim]
bioclim = [x.T for x in bioclim]
bioclim = [x[environ_cols] for x in bioclim]
bioclim = [x.astype(float) for x in bioclim]
bioclim = [x.apply(preprocessing.scale, axis=1) for x in bioclim]

In [0]:
bioclim = [x.assign(blank=lambda x: "") for x in bioclim]

In [0]:
bioclim.append(bioclim[0])

In [0]:
for i,snps in enumerate(snpsfile_df):
    snp_outfile = os.path.join(analysis_dir[i], "snpsfile")
    env_outfile = os.path.join(analysis_dir[i], "environfile")
    snps.to_csv(snp_outfile, sep="\t", header=False, index=False)
    envs = bioclim[i]
    envs.to_csv(env_outfile, sep="\t", header=False, index=True)

## Run bayenv to create covariance matrix

```bash
~/g/src/bayenv2_public/bayenv2 -i snpsfile -p 6 -k 100000 -r 187564 > matrix.out
```

## Check cov matrix estimation

In [0]:
with open(os.path.join(analysis_dir[1], "bayenv_cov.txt"), "w") as o:
    for i in range(0, 5):
        r = random.getrandbits(16)
        cmd = "~/g/src/bayenv2_public/bayenv2 -i snpsfile -p 6 -k 100000 -r %g > matrix_%d.out" % (r, i)
        o.write("%s\n" % cmd)

In [0]:
with open(os.path.join(analysis_dir[1], "bayenv_cov_1M.txt"), "w") as o:
    for i in range(0, 5):
        r = random.getrandbits(16)
        cmd = "~/g/src/bayenv2_public/bayenv2 -i snpsfile -p 6 -k 1000000 -r %g > matrix_1M_%d.out" % (r, i)
        o.write("%s\n" % cmd)

In [0]:
with open(os.path.join(analysis_dir[1], "bayenv_cov_500k.txt"), "w") as o:
    for i in range(0, 5):
        r = random.getrandbits(16)
        cmd = "~/g/src/bayenv2_public/bayenv2 -i snpsfile -p 6 -k 500000 -r %g > matrix_500k_%d.out" % (r, i)
        o.write("%s\n" % cmd)

In [0]:
basedir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/"

In [0]:
def read_mat(m):
    vcovs = []
    current = None
    for line in open(m):
        if "VAR-COVAR" in line:
            current = []
            vcovs.append(current)
        if isinstance(current, list):
            current.append(line.strip().split("\t"))
    return vcovs

In [0]:
mats = {}
for i in range(0,5):
    fname = os.path.join(basedir, "matrix_%d.out" % i)
    mats[i] = read_mat(fname)

mat_dfs = {}
for m in mats:
    temp = []
    for i, elem in enumerate(mats[m]):
        temp.append(pd.DataFrame(elem[1:]).T)
    mat_dfs[m] = temp

In [0]:
bayenv_hm = pd.DataFrame(np.zeros((len(mat_dfs),len(mat_dfs))))
for i in mat_dfs:
    idf = mat_dfs[i][-1].drop(6, axis=1)
    idf = [float(x) for x in idf.values.flatten()]
    for j in mat_dfs:
        jdf = mat_dfs[j][-1].drop(6, axis=1)
        jdf = [float(x) for x in jdf.values.flatten()]
        bayenv_hm.ix[i,j] = sp.stats.pearsonr(idf, jdf)[0]

In [0]:
final_mats = []
for i in mat_dfs:
    final_mats.append(mat_dfs[i][-1].drop(6, axis=1))

In [0]:
mean_mat = pd.DataFrame(index=final_mats[0].index, columns=final_mats[1].columns)
for i in range(len(mean_mat)):
    for j in range(len(mean_mat)):
        vals = []
        for k in range(len(final_mats)):
            vals.append(final_mats[k].ix[i, j])
        vals = [float(x) for x in vals]
        mean_mat.ix[i,j] = np.mean(vals)
        

In [0]:
def mat_corr(idf, jdf):
    idf = [float(x) for x in idf.values.flatten()]
    jdf = [float(x) for x in jdf.values.flatten()]
    return sp.stats.pearsonr(idf, jdf)
    

In [0]:
for fm in final_mats:
    print(mat_corr(mean_mat, fm))

In [0]:
out_matrix = os.path.join(basedir, "matrix_chain_avg.out")
mean_mat.to_csv(out_matrix, sep="\t", index=False, header=False)

In [0]:
snp_indv = [x.drop("blank", axis=1) for x in snpsfile_df]

In [0]:
for s in snp_indv:
    s['snp_name'] = s.apply(lambda x: "_".join(x.name.split("_")[0:-1]), axis=1)

In [0]:
%connect_info

In [0]:
def write_bayenv_files(dirname, snp_indv_data):
    bayenv = []
    for i, s in enumerate(snp_indv_data):
        if i == 0:
            continue            
        temp = []
        bayenv.append(temp)
        outdir = os.path.join(analysis_dir[i], dirname)
        if not os.path.exists(outdir):
            os.makedirs(outdir, exist_ok=True)
        for group, data in s.groupby("snp_name"):
            bayenv_file = os.path.join(outdir, "%s.txt" % group)
            temp.append(bayenv_file)
            data = pd.DataFrame(data)
            data['blank'] = ""
            data = data.drop("snp_name", axis=1)
            data.to_csv(bayenv_file, index=False, header=False, sep="\t")
    return bayenv

In [0]:
bayenv = write_bayenv_files("bayenv_runs/infiles", snp_indv)

In [0]:
bayenv = !find {os.path.join(analysis_dir[1], "bayenv_runs/infiles")} -name "*.txt"

In [0]:
bayenv = [bayenv]

In [0]:
bayenv.insert(0, [])

In [0]:
len(bayenv)

In [0]:
bayenv_jobs = 150

In [0]:
num_chains = 5

In [0]:
num_chunks = int(np.round(len(bayenv[1])/bayenv_jobs))

In [0]:
num_chunks

In [0]:
bayenv_rundir = os.path.join(analysis_dir_imp, "bayenv_runs")

In [0]:
def make_bayenv_dirs(base, n):
    for i in range(n):
        os.makedirs(os.path.join(base, "chain_%d" % i), exist_ok=True)


make_bayenv_dirs(bayenv_rundir, num_chains)

In [0]:
ngen = 5e5

In [0]:
def get_bayenv_seed():
    s = int(random.getrandbits(16))
    if s == 0:
        return get_bayenv_seed()
    return s

In [0]:
wrapper = "ipython ~/ipynb/gypsy_moth/run_bayenv.ipy"
bayenv_exe = "/home/cfriedline/g/src/bayenv2_public/bayenv2"
bayenv_opt = "-i snpfile -m matrixfile -e environfile -p 6 -k 100000 -n 19 -t -X -c -f -r rand"
matrix_file = "matrix_chain_avg.out"
environfile = "environfile"
for i, bayenv_files in enumerate(bayenv):
    if i == 0:
        continue
    thedir = analysis_dir[i]
    shutil.copy(os.path.join(thedir, matrix_file), os.path.dirname(bayenv_files[0]))
    shutil.copy(os.path.join(thedir, environfile), os.path.dirname(bayenv_files[0]))
    chunks=[bayenv_files[x:x+num_chunks] for x in range(0, len(bayenv_files), num_chunks)]
    
    for i, chunk in enumerate(chunks):
        with open(os.path.join(os.path.dirname(bayenv_files[0]), "bayenv_parallel_%d" % i), "w") as o:
            for bayenv_file in chunk:
                for chain in range(num_chains):
                    outdir = os.path.join(bayenv_rundir, "chain_%d" % chain)
                    bayenv_cmd = " ".join([bayenv_exe, bayenv_opt]).split()
                    bayenv_cmd[2] = os.path.basename(bayenv_file)
                    bayenv_cmd[4] = matrix_file
                    bayenv_cmd[6] = environfile
                    bayenv_cmd[-1] = get_bayenv_seed()
                    bayenv_cmd[10] = int(ngen)
                    outfile = os.path.basename(bayenv_file).replace(".txt","")
                    bayenv_cmd.append("-o %s" % os.path.join(outdir, outfile))
                    o.write("%s '" % wrapper)
                    o.write("%s" % " ".join([str(x) for x in bayenv_cmd]))
                    o.write("'\n")

In [0]:
def write_qsub_files(bayenv_parallel, dirname):
    files = []
    for i, f in enumerate(bayenv_parallel):
        d = os.path.join(analysis_dir[1], dirname)
        qsub_file = os.path.join(d, "qsub_%d.sh" % i)
        files.append(qsub_file)
        with open(qsub_file, "w") as o:
            os.chmod(o.name, 0o744)
            #print(o.name)
            o.write("%s\n" % "\n".join(["#!/bin/bash", 
                                        "#$ -N bayenv%d" % i,
                                        "#$ -V",
                                        "#$ -cwd",
                                       "#$ -pe smp 30",
                                       "#$ -j y",
                                        "#$ -q all.q",
                                        "#$ -e ../bayenv%d.err" % i,
                                        "#$ -o ../bayenv%d.out" % i,
                                        "unset module",
                                        "echo \"Running on $HOSTNAME\"",
                                       "cat %s | ~/bin/parallel -j 30 --delay 1 --" % f]))
    return files

In [0]:
bayenv_infile_dir = os.path.join(bayenv_rundir, "infiles")

In [0]:
bayenv_parallel = !ls {bayenv_infile_dir} | grep parallel

In [0]:
len(bayenv_parallel)

In [0]:
qsub_files = write_qsub_files(bayenv_parallel, bayenv_infile_dir)

In [0]:
cat /home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/bayenv_runs/infiles/qsub_0.sh

In [0]:
for i, ad in enumerate(analysis_dir):
    if i == 0:
        continue
    d = bayenv_infile_dir
    with open(os.path.join(d, "qsub_runner.sh"), "w") as o:
        os.chmod(o.name, 0o744)
        o.write("#!/bin/bash\n")
        o.write("unset module\n")
        for q in qsub_files:
            o.write("qsub %s\n" % q)

In [0]:
!head -n5 {os.path.join(d, "qsub_runner.sh")}

In [0]:
qhost = !qhost | grep godel

In [0]:
for elem in qhost:
    host = elem.split()[0]
    if not host in ['godel200',
                   'godel21',
                   'godel37']:
        !ssh {host} pkill -9 bayenv

## Run bayenv jobs

```bash
cd ~/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/bayenv_runs/infiles
./qsub_runner.sh
```

In [0]:
bayenv_rundir

In [0]:
def check_progress():
    progress = {}
    for i in range(num_chains):
        d = os.path.join(bayenv_rundir, "chain_%d" % i)
        files = [os.path.join(d, x) for x in scandir.listdir(d) if ".bf" in x]
        print(d, len(files))
        for f in files:
            name = os.path.basename(f)
            chain = os.path.basename(os.path.dirname(f))
            if not name in progress:
                progress[name] = []
            progress[name].append(chain)
    return progress

def progress_stats(progress):
    c = Counter()
    inprogress = set()
    for snp in progress:
        if len(progress[snp]) < 5:
            inprogress.add(snp)
        c[len(progress[snp])] += 1
    return c, sorted(inprogress), progress

progress = check_progress()
c, inprog, prog = progress_stats(progress)

In [0]:
parallel_files = !ls {os.path.join(bayenv_rundir, "infiles")}/bayenv_parallel*

In [0]:
def get_parallel_file(snp):
    snp = snp.replace(".bf", "")
    cmd = "grep %s %s/bayenv_parallel*" % (snp, bayenv_infile_dir)
    res = !$cmd
    return res

leftovers = []
for snp in inprog:
    completed = prog[snp]
    res = get_parallel_file(snp)
    for elem in res:
        f, cmd = elem.split(":")
        chain = os.path.basename(os.path.dirname(cmd.split()[-1]))
        if chain not in completed:
            leftovers.append(cmd)

In [0]:
leftovers, len(leftovers)

## random seed fixing

for some reason (me), some random seeds were randomly set to zero, let's fix that.

In [0]:
bayenv_makeup = os.path.join(bayenv_infile_dir, "bayenv_makeup.txt")
with open(bayenv_makeup, "w") as o:
    for elem in leftovers:
        cmd = elem.split()
        if (int(cmd[20])) == 0:
            cmd[20] = str(get_bayenv_seed())
        print(cmd)
        elem = " ".join(cmd)
        print(elem)
        o.write("%s\n" % elem)

## Bayenv makeup job

```
#!/bin/bash
# makeup.sh
#$ -N bayenv_makeup
#$ -V
#$ -cwd
#$ -pe smp 30
#$ -j y
#$ -q all.q
#$ -e ../bayenv_makeup.err
#$ -o ../bayenv_makeup.out
unset module
echo "Running on $HOSTNAME"
cat bayenv_makeup.txt | ~/bin/parallel -j 30 --delay 1 --
```

```
cd /gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/bayenv_runs/infiles
qsub makeup.sh
```

In [0]:
len(leftovers), c, inprog

In [0]:
vartypes = ['bf', 'rho', 'rs']
bf_cols = []
for b in bioclim[0].index:
    for v in vartypes:
        bf_cols.append("%s_%s" % (b, v))

In [0]:
chain_dirs = [os.path.join(bayenv_rundir, "chain_%d" % x) for x in range(5)]

In [0]:
xtxs = {}
bfs = {}
for d in chain_dirs:
    name = os.path.basename(d)
    files = [os.path.join(d, x) for x in scandir.listdir(d)]
    xtxs[name] = [x for x in files if x.endswith(".xtx")]
    bfs[name] = [x for x in files if x.endswith(".bf")]

In [0]:
[len(bfs[x]) for x in bfs]

In [0]:
rc = Client(profile="sge")
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
def read_bayenv(args):
    f, key, cols, chain = args
    import pandas as pd
    key = key.lower()
    df = pd.read_csv(f, sep="\t", header=None, index_col=0)
    if key == 'xtx':
        df.columns = ['xtx']
    elif key == "bf":
        df = df.drop(df.columns[-1], axis=1)
    df.index = ["%s-%s" % (x.replace(".txt", ""), chain) for x in df.index]
    df.index.name = "SNP"
    if cols:
        df.columns=cols
    return df.ix[0,:]

In [0]:
dv['read_bayenv'] = read_bayenv

In [0]:
bf_args = []
for chain in bfs:
    for x in bfs[chain]:
        bf_args.append((x, 'bf', bf_cols, chain))

In [0]:
len(bf_args)

In [0]:
bf_dfs = dv.map_async(read_bayenv, bf_args)

In [0]:
bf_dfs.progress, bf_dfs.ready()

In [0]:
bf_dfs = bf_dfs.r

In [0]:
analysis_dir[1]

In [0]:
bf = pd.concat([pd.DataFrame(x).T for x in bf_dfs])

In [0]:
bf.shape

In [0]:
analysis_dir

In [0]:
sns.set_context("notebook")

In [0]:
ax = sns.distplot(xtx)
ax.set_xlabel("XtX")
plt.show()

In [0]:
joined = xtx.join(bf)

In [0]:
joined.head()

In [0]:
joined.shape

In [0]:
bayenv_dir = os.path.join(analysis_dir_imp, "bayenv")
joined.to_csv(os.path.join(bayenv_dir, "bayenv_results.txt"), 
             sep="\t", 
             header=True,
             index=True)

In [0]:
bayenv_dir

In [0]:
def plot_bf_vs_xtx(df, imputed, dir_name):
    num_figs = len(df.columns)-1
    print(num_figs)
    plt.gcf().set_size_inches(20,20)
    for i in range(num_figs):
        if i > 0:
            plt.subplot(5,4,i)
            plt.title("%s (%s)" % (df.columns[i].split("_")[0], imputed))
            plt.xlabel("BF")
            plt.ylabel(df.columns[0])
            plt.scatter(df.ix[:,i],df.ix[:,0])

    plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                    wspace=.5, hspace=.5)
    plt.savefig("%s.pdf" % os.path.join(dir_name, "bf_vs_xtx_%s" % imputed))
    plt.show()
plot_bf_vs_xtx(pd.DataFrame(xtx).join(bf[[x for x in bf if 'bf' in x]]), "imputed", analysis_dir[1])

In [0]:
med_xtx = np.median(joined.xtx)

In [0]:
mad = np.median(np.abs(joined.xtx-med_xtx))

In [0]:
mad

In [0]:
xtx_outliers = []
impute_status = ["imputed"]
for i, res in enumerate([joined]):
    XtX = res.xtx
    M = np.median(XtX)    
    MAD = sm.robust.mad(XtX)
    lower_cutoff = M-(3*MAD)
    upper_cutoff = M+(3*MAD)
    lower_snps = sorted(XtX[XtX<lower_cutoff].index.tolist())
    upper_snps = sorted(XtX[XtX>upper_cutoff].index.tolist())
    percent_cutoff = np.percentile(XtX, 99)
    percent_cutoff_outliers = sorted(XtX[XtX > percent_cutoff].index.tolist())
    o = pd.DataFrame([impute_status[i], 
                      M,
                      MAD,
                      lower_cutoff, 
                      upper_cutoff, 
                      len(lower_snps), 
                      len(upper_snps), 
                      lower_snps, 
                      upper_snps,
                     percent_cutoff, 
                      len(percent_cutoff_outliers),
                      percent_cutoff_outliers]).T
    o.columns = ['state', 'median_XtX', 'MAD', 'lower_cutoff', 'upper_cutoff', 'n_lower', 'n_upper', 
                'lower_snps', 'upper_snps', 'percent_cutoff_XtX', 'n_percent_cutoff', '99th_snps']
    xtx_outliers.append(o)

In [0]:
pd.concat(xtx_outliers)

In [0]:
joined.head()

In [0]:
joined.shape

In [0]:
env_outliers = {}
env_cols = [x for x in joined if '_bf' in x or '_rho' in x]
env_df = joined[env_cols]

In [0]:
for bio, data in env_df.groupby(lambda x: x.split("_")[0], axis=1):
    env_outliers[bio] = []
    cutoff = data.apply(lambda x: x.quantile(0.99))
    for c in cutoff.index:
        outliers = set(data[data[c] >= cutoff[c]].index.tolist())
        env_outliers[bio].append(outliers)

In [0]:
env_outliers_isect = env_outliers.copy()

In [0]:
for bio in env_outliers_isect:
    snplist = env_outliers_isect[bio]
    env_outliers_isect[bio] = snplist[0].intersection(snplist[1])

In [0]:
with open(os.path.join(bayenv_dir, "env_outliers_isect.pkl"), "wb") as o:
    pickle.dump(env_outliers_isect, o, pickle.HIGHEST_PROTOCOL)

---
# Bayenv convergence testing

In [0]:
joined_bf = joined[[x for x in joined if '_bf' in x]]

In [0]:
high_bf_test = joined_bf[joined_bf.BIO10_bf > 100]

In [0]:
high_bf_test

In [0]:
original_seeds = {
    "jtg7180005921436f_7180005479371f_7180005921437f_7180005921438f_5770": 17046,
    "jtg7180005937218f_7180005937219f_4892": 54214
}

In [0]:
snp_indv_test = snp_indv.copy()
snp_indv_test[0] = []

In [0]:
snp_indv_test[1] = snp_indv_test[1][snp_indv_test[1].snp_name.isin(high_bf_test.index)]

In [0]:
test_files = write_bayenv_files("bayenv_test", snp_indv_test)
test_files.insert(0, [])

In [0]:
test_files

In [0]:
original_seeds

In [0]:
bayenv_exe = "/home/cfriedline/g/src/bayenv2_public/bayenv2"
bayenv_opt = "-i snpfile -m matrixfile -e environfile -p 6 -k 100000 -n 19 -t -X -c -f -r rand -o outfile"

max_cpu = 30
cpu = 0
for i, bayenv_files in enumerate(test_files):
    if i == 0:
        continue
    thedir = analysis_dir[i]
    shutil.copy(os.path.join(thedir, "matrix_last.out"), os.path.dirname(bayenv_files[0]))
    shutil.copy(os.path.join(thedir, "environfile"), os.path.dirname(bayenv_files[0]))
    chunks=[bayenv_files[x:x+num_chunks] for x in range(0, len(bayenv_files), num_chunks)]
    for i, chunk in enumerate(chunks):
        with open(os.path.join(os.path.dirname(bayenv_files[0]), "bayenv_parallel_%d" % i), "w") as o:
            for bayenv_file in chunk:
                seed = int(random.getrandbits(16))
                for i in range(100000, 10200000, 500000):
                    if cpu == max_cpu:
                        cpu = 0
                    bayenv_cmd = " ".join([bayenv_exe, bayenv_opt]).split()
                    bayenv_cmd[2] = os.path.basename(bayenv_file)
                    bayenv_cmd[4] = "matrix_last.out"
                    bayenv_cmd[6] = "environfile"
                    #bayenv_cmd[18] = int(random.getrandbits(16)) # set random seed
                    #bayenv_cmd[18] = original_seeds[os.path.basename(bayenv_file).replace(".txt","")]
                    bayenv_cmd[18] = seed
                    bayenv_cmd[14] = "" #discard xtx
                    bayenv_cmd[10] = i
                    bayenv_cmd[-1] = "%s-%d" % (os.path.basename(bayenv_file).replace(".txt",""), i)
                    bayenv_cmd.insert(0, "taskset -c %d" % cpu)
                    o.write("%s\n" % " ".join([str(x) for x in bayenv_cmd]))   
                    cpu += 1

In [0]:
test_parallel = !find {os.path.join(analysis_dir[1], "bayenv_test")} | grep parallel_

In [0]:
test_parallel

In [0]:
write_qsub_files(test_parallel, "bayenv_test")

## run bayenv

```
qsub qsub_0.sh
```

Notes:
* bayenv_test_3 - original fixed seed, change generations
* bayenv_test_2 - change seed for each # of generations
* bayenv_test_1 - change seed for each # of generations


In [0]:
def read_test_bayenv(args):
        f, key, cols = args
        import pandas as pd
        key = key.lower()
        df = pd.read_csv(f, sep="\t", header=None, index_col=0)
        if key == 'xtx':
            df.columns = ['xtx']
        elif key == "bf":
            df = df.drop(df.columns[-1], axis=1)
        base = os.path.basename(f).replace(".bf", "")
        base_data = base.split("-")
        assert len(df) == 1
        df.index = [base]
        df.index.name = "SNP"
        if cols:
            df.columns=cols

        df['snp'] = base_data[0]
        df['ngen'] = int(base_data[1])
        return df.ix[0,:]

all_tests = {}
    
for test_dir in ["bayenv_test", "bayenv_test_1", "bayenv_test_2", "bayenv_test_3"]:
    test_bfs = !ls {analysis_dir[1]}/{test_dir} | grep '.bf'
    test_bfs = sorted([os.path.join(analysis_dir[1] + test_dir, x) for x in test_bfs])

    test_bf_args = [(x, 'bf', bf_cols) for x in test_bfs]

    test_bfs = []
    for x in test_bf_args:
        test_bfs.append(read_test_bayenv(x))

    test_joined = pd.concat([pd.DataFrame(x).T for x in test_bfs])[['BIO10_bf', 'snp', 'ngen']]
    all_tests[test_dir] = test_joined
    for snp, data in test_joined.groupby('snp'):
        sns.regplot("ngen", "BIO10_bf", data=data, fit_reg=False)
        plt.title("%s (%s)" % (snp, test_dir))
        plt.show()

In [0]:
for key in all_tests:
    d = all_tests[key]
    d.index = ["%s-%s" % (key, x) for x in d.index]

In [0]:
all_df = pd.concat([all_tests[x] for x in all_tests])

In [0]:
all_df['test'] = all_df.apply(lambda x: x.name.split("-")[0], axis=1)

In [0]:
box_data = {}
for key, data in all_df.groupby(("snp")):
    g = sns.boxplot(x="ngen", y="BIO10_bf", data=data)
    g.set_yscale('log')
    plt.xticks(rotation=90)
    plt.title(key)
    plt.show()